# Imports

*notes:*
- change this to be a normal python file

In [ ]:
import asyncio
import subprocess
import pandas as pd

import strym
print(strym.__version__)

# Functions from analyse_and_create_db.ipynb

In [ ]:
async def async_command_shell(command, verbose: bool = False):
    """Run command in subprocess (shell).
    source: https://fredrikaverpil.github.io/2017/06/20/async-and-await-with-subprocesses/
    """
    # Create subprocess
    process = await asyncio.create_subprocess_shell(command, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
    # Status
    if verbose:
        print("Started:", command, "(pid = " + str(process.pid) + ")", flush=True)
    # Wait for the subprocess to finish
    stdout, stderr = await process.communicate()
    # Output
    if process.returncode == 0:
        if verbose:
            print("Done:", command, "(pid = " + str(process.pid) + ")", flush=True)
        return stdout.decode().strip()
    else:
        if verbose:
            print("Failed:", command, "(pid = " + str(process.pid) + ")", flush=True)
        raise Exception(stderr.decode().strip())


async def iget(file_adress, destination, verbose: bool = False):
    '''
    wrapper for iRODS iget command
    async command using asyncio library
    :param file_adress: address on CyVerse fileshare
    :param destination: address to download to on the local computer
    :return: local address of the file
    '''
    try:
        await async_command_shell(f'iget -T {file_adress} {destination}', verbose=verbose)
        local_address = destination + '/' + file_adress.split('/')[-1]
        return local_address
    except Exception as e:
        raise Exception(f'Error while downloading file at:'
                        f'\n\tremote: {file_adress}'
                        f'\n\tto local address: {destination}`'
                        f'\n\tFailing on {e}')


def init_cache(local_folder):
    '''
    clears the cache if exists and initialise it
    :param local_folder: root folder for the analysis
    :return: temporary cache address
    '''
    subprocess.run(['cd', local_folder],
                   stdout=subprocess.PIPE,
                   stderr=subprocess.PIPE,
                   universal_newlines=True)
    local_folder_absolute = subprocess.run(['pwd'],
                   stdout=subprocess.PIPE,
                   stderr=subprocess.PIPE,
                   universal_newlines=True).stdout.strip()
    files = subprocess.run(['ls'],
                   stdout=subprocess.PIPE,
                   stderr=subprocess.PIPE,
                   universal_newlines=True)
    files = files.stdout.split(sep='\n')
    if 'temp_cache' in files:
        subprocess.run(['rm', '-r', '-f', 'temp_cache'],
                   stdout=subprocess.PIPE,
                   stderr=subprocess.PIPE,
                   universal_newlines=True)
    subprocess.run(['mkdir', 'temp_cache'],
               stdout=subprocess.PIPE,
               stderr=subprocess.PIPE,
               universal_newlines=True)
    temp_cache_address = f'{local_folder_absolute}/temp_cache'
    return temp_cache_address


# Download and serve file

In [ ]:
class FileServer:
    """
    Class handling download and delete of files to be sent to ROS playback
    """
    # attributes
    data = None
    current_remote_adresses = None
    current_event = None
    can_local_address = None
    gps_local_address = None
    # note: first file served after incrementation
    index = 0
    max_index = None

    # methods
    def __init__(self, analysis_path):
        """
        Reads from a CSV analysis file
        :param analysis_path: string of the path of the CSV taken from the analysis
        """
        self.data = pd.read_csv(analysis_path)

    def filter(self, cc_state: [int], speed: {str: int}, vin: [str], date: {str: str}, event_type: [str]):
        """
        filter the rows based on those criteria
        :param cc_state: list of acceptable controller state values
        :param speed: {min: int min_speed in km/h, max: int max_speed in km/h}
        :param vin: list of acceptable vehicle identification numbers
        :param date: {beg: date, end: date}, with date as strings, formatted as YYYY-MM-DD-HH-MM-SS
        :param event_type: list of acceptable event types. possible event types are:
            - car_crossing
            - <more to come in the future>
        :return: updates self.data to only keep the desirable instances
        """
        self.data = self.data.loc[self.data['event_type'] in event_type]
        self.data = self.data.loc[self.data['vin'] in vin]
        self.data = self.data.loc[self.data['event_cc_state'] in cc_state]
        self.data = self.data.loc[(self.data['event_speed'] >= speed['min'])
                                  & (self.data['event_speed'] >= speed['min'])]
        self.data = self.data.loc[(self.data['date'] >= date['min'])
                                  & (self.data['date'] >= date['min'])]
        self.max_index = len(self.data)

    def __str__(self):
        if self.max_index is None:
            return f'FileServer filtering is not finished'
        else:
            return f'file server with {self.max_index} files ready to be served, current index is: {self.index}'

    async def next(self, ignore_gps_file: bool = False):
        """
        clears cache & downloads the next couple of files
        :param: ignore_gps_file: set to True to avoid downloading the GPS file
        :return: - object with paths to the downloaded CAN and GPS file
        {'can': str, 'gps': str, 'remote_addresses': {'can': str, 'gps': str}}
                 - if the maximum index is reached, returns an exception as:
        Exception('max_index')
        """
        try:
            if self.index < self.max_index:
                cache = init_cache('database_exploration')
                self.current_event = self.data[self.index]
                self.current_remote_adresses = self.current_event['remote_addresses']

                self.can_local_address = await iget(self.current_remote_adresses['can'], cache)
                if ignore_gps_file:
                    self.gps_local_address = None
                else:
                    self.gps_local_address = await iget(self.current_remote_adresses['gps'], cache)

                self.index += 1

                return {
                    'can': self.can_local_address,
                    'gps': self.gps_local_address,
                }
            else:
                raise Exception('max_index')

        except Exception as e:
            raise Exception(f'Downloading next file failed on {e}')

    def clear(self):
        cache = init_cache('database_exploration')
        print('Cache cleared')


# Can to bagfile

# 1 file serving (automate this with an inner state of file handler out of a notebook?)
### handle here the file cutting?